In [ ]:
import glob
import re
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from functools import reduce

In [ ]:
consumer = dict()

for file in glob.glob("./logs/consumer_producer_*.txt"):
    x = pd.read_csv(file, sep=" ", names=['type', 'time', 'r_count', 's_count', 'id'])
    x.time = pd.to_datetime(x.time, format="%H:%M:%S:%f")
    x.r_count = pd.to_numeric(x.r_count.str.replace("R=", ""))
    x.s_count = pd.to_numeric(x.s_count.str.replace("S=", ""))
    x.id = pd.to_numeric(x.id.str.replace("id=", ""), errors='coerce')
    file = file.replace('./logs/', '').replace('.txt', '')
    consumer.update({file: x})

In [ ]:
list(consumer.keys())

In [ ]:
consumer_counts = [(key, int(value.r_count.tail(1)), int(value.s_count.tail(1))) for key, value in consumer.items()]
consumer_counts = pd.DataFrame(consumer_counts, columns=['name', 'R', 'S'])
consumer_counts['ratio'] = consumer_counts.S / consumer_counts.R
consumer_counts

In [ ]:
consumer_counts.describe()

In [ ]:
consumer_counts.set_index('name').R.plot(kind='bar')

In [ ]:
def get_usage(dict_data):
    dict_data = consumer
    usage = {key: value.query("type=='R'").set_index('time').r_count.resample("1s").count() for key, value in dict_data.items()}
    min_time = min([value.head(1).index[0] for value in usage.values()])
    max_time = max([value.tail(1).index[0] for value in usage.values()])
    usage_pre = {key: pd.Series(0, pd.date_range(start=min_time, end=value.head(1).index[0] - datetime.timedelta(seconds=1), freq="1s")) if value.head(1).index[0] > min_time else None for key, value in usage.items()}
    usage_post = {key: pd.Series(0, pd.date_range(start=value.tail(1).index[0] + datetime.timedelta(seconds=1), end=max_time, freq="1s")) if value.tail(1).index[0] < max_time else None for key, value in usage.items()}
    usage = {key: pd.concat([usage_pre[key], value, usage_post[key]]) for key, value in usage.items()}
    return(usage)


In [ ]:
consumer_usage = get_usage(consumer)
consumer_usage

In [ ]:
consumer_usage['consumer_producer_8'].plot()

In [ ]:
palette = plt.get_cmap('Set1')

i = 1
for key, value in consumer_usage.items():
    plt.plot(value.rolling(60).mean(), marker='', color=palette(i), linewidth=1, alpha=0.9, label=key)
    i+=1

In [ ]:
{key: ((value>0)*1).mean() for key, value in consumer_usage.items()}

In [ ]:
sum(consumer_usage.values()).plot()

In [ ]:
sum(consumer_usage.values()).rolling(10).mean().plot()

In [ ]:
reduce(lambda x, value:x + (value>0)*1, consumer_usage.values(), 0).plot()

In [ ]:
reduce(lambda x, value:x + (value>0)*1, consumer_usage.values(), 0).rolling(30).mean().plot()